In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 20963216
paper_name = 'ratnakumar_oliver_2011' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['pmid', 'name'])

In [4]:
datasets.set_index('pmid', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/c0mb00114g.xls', sheet_name='A5', skiprows=1)

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 653 x 4


In [7]:
orf_col = 'Systematic ID'

In [8]:
original_data[orf_col] = original_data[orf_col].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data[orf_col] = clean_orf(original_data[orf_col])

In [10]:
# Translate to ORFs 
original_data['orfs'] = translate_sc(original_data[orf_col], to='orf')

In [11]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orfs'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [Systematic ID, Gene name, Fold-change, Description, orfs]
Index: []


In [12]:
# Transform into TRT/UNT ratio, and take the log2
original_data['data'] = np.log2(-1/original_data['Fold-change'])

In [13]:
original_data.set_index('orfs', inplace=True)

In [14]:
original_data.index.name = 'orf'

In [15]:
original_data = original_data[['data']].copy()

In [16]:
original_data = original_data.apply(pd.to_numeric, axis=1, errors='coerce')

In [17]:
original_data = original_data.groupby(original_data.index).mean()

In [18]:
original_data.shape

(651, 1)

# Prepare the final dataset

In [19]:
data = original_data.copy()

In [20]:
dataset_ids = [16474]
datasets = datasets.reindex(index=dataset_ids)

In [21]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [22]:
data.head()

dataset_id,16474
data_type,value
orf,
YAL002W,-5.828952
YAL019W,-3.269248
YAL026C,-5.562175
YAL039C,-8.095157
YAL044C,-6.241994


## Subset to the genes currently in SGD

In [23]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [24]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,16474
,data_type,value
gene_id,orf,
2,YAL002W,-5.828952
17,YAL019W,-3.269248
24,YAL026C,-5.562175
37,YAL039C,-8.095157
42,YAL044C,-6.241994


# Normalize

In [25]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [26]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [27]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id          16474          
data_type           value    valuez
gene_id orf                        
2       YAL002W -5.828952 -3.681769
17      YAL019W -3.269248 -2.064971
24      YAL026C -5.562175 -3.513263
37      YAL039C -8.095157 -5.113183
42      YAL044C -6.241994 -3.942661

# Print out

In [28]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [29]:
from IO.save_data_to_db3 import *

In [30]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 20963216...
Inserting the new data...


100%|██████████| 1/1 [00:01<00:00,  1.11s/it]

Updating the data_modified_on field...
